**Installing Wandb**

In [ ]:
!pip install wandb

**Import statements**

In [2]:
from keras.datasets import fashion_mnist
import wandb
import numpy as np

**Load dataset**

In [ ]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

# **Question 1**

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
k = len(class_names)
wandb.init(
    project="Assignment 1",
)
def plotImagesOfEachClass():
  image_labels = []
  images = []
  for i in range(len(trainX)):
    if len(image_labels) == len(class_names):
      break
    if class_names[trainy[i]] not in image_labels:
      image_labels.append(class_names[trainy[i]])
      images.append(trainX[i])

  wandb.log({"examples ": [wandb.Image(img, caption=caption) for img, caption in zip(images, image_labels)]})

plotImagesOfEachClass()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# **Question 2**

In [ ]:
a=np.random.rand(2)
a.dtype

dtype('float64')

In [12]:
def initializeWeightAndBias(num_hidden_layers, neurons_in_each_layer, num_images, k, image_size):
  W = []
  bias = []
  W.append(np.random.uniform(-0.72, 0.72, (neurons_in_each_layer, 784)))
  bias.append((np.random.uniform(-0.72, 0.72, (neurons_in_each_layer,1))))
  for i in range(num_hidden_layers-1):
    W.append((np.random.uniform(-0.72, 0.72, (neurons_in_each_layer, neurons_in_each_layer))))
    bias.append((np.random.uniform(-0.72, 0.72, (neurons_in_each_layer,1))))
  W.append(np.random.uniform(-0.72, 0.72, (k, neurons_in_each_layer)))
  bias.append(np.random.uniform(-0.72, 0.72, (k,1)))
  return W, bias

def flattenList(X):
  return X.flatten()

def sigmoid(X):
  return 1.0/(1.+np.exp(-X))

def softmax(a):
  return np.exp(a)/np.sum(np.exp(a))

def feedForward(W, bias, X, num_hidden_layers, neurons_in_each_layer):
  preactivation = []
  activation = []
  activation.append(X.reshape(784,1))
  preactivation.append(X.reshape(784,1))
  for i in range(1, num_hidden_layers+1):
    preactivation.append(np.add(bias[i-1], np.matmul(W[i-1], activation[(i-1)])))
    activation.append(sigmoid(preactivation[i]))
  preactivation.append(np.add(bias[-1], np.dot(W[-1], activation[-1])))
  activation.append(softmax(preactivation[-1]))
  return activation, preactivation

In [6]:
def updateParam(W, gradientW, bias, gradientBias, L, eta):
  for i in range(0, len(W)):
    W[i] = W[i] - eta*gradientW[i]
    bias[i] = bias[i] - eta*gradientBias[i]
  return W, bias

In [7]:
def derivative_sigmoid(x):
  return sigmoid(x)*(np.ones_like(x)-sigmoid(x))
def der_softmax(x):
  return softmax(x)*(np.ones_like(x)-softmax(x))

In [20]:
def backward_propogation(x, y, W, bias, activation, preactivation, num_hidden_layers, neurons_in_each_layer, k, num_images, batchSize):
  L = num_hidden_layers+1
  gradientActivation = []
  gradientPreactivation = []
  exp_y = np.zeros((k,1))
  exp_y[y][0] = 1
  gradientPreactivation.append(-(exp_y-activation[L]))
  gradientWeight = []
  gradientBias = []

  for k in range(L, 0, -1):
    gradientWeight.append(np.matmul(gradientPreactivation[-1], activation[k-1].T)/batchSize)
    gradientBias.append(gradientPreactivation[-1]/batchSize)
    if k==1:
      break
    gradientActivation.append(np.matmul(W[k-1].T, gradientPreactivation[-1]))
    gradientPreactivation.append(np.multiply(gradientActivation[-1], derivative_sigmoid(preactivation[k-1])))
  return gradientWeight[::-1], gradientBias[::-1]

In [9]:
def one_hot(i):
  y = np.zeros((10,1))
  y[i] = 1
  return y

def cross_entropy(true_output, predicted_output):
  return -1.0*np.sum(true_output*np.log(predicted_output))

In [16]:
def stochastic_gradient_descent(num_hidden_layers, neurons_in_each_layer, num_images, image_size, trainX, trainY, epochs, eta):
  W, bias = initializeWeightAndBias(num_hidden_layers, neurons_in_each_layer, num_images, k, image_size)
  batchSize = 32
  y_pred = []
  print(epochs)
  for iterationNumber in range(epochs):
    gradientW = []
    gradientBias = []
    
    correctPrediction = 0
    print(iterationNumber)
    loss=0
    count=0
    for i in range(num_images):
      activation, preactivation = feedForward(W, bias, trainX[i], num_hidden_layers, neurons_in_each_layer)
      loss += cross_entropy(one_hot(trainy[i]), activation[num_hidden_layers+1])
      gradientW, gradientBias = backward_propogation(trainX[i], trainY[i], W, bias, activation, preactivation, num_hidden_layers, neurons_in_each_layer, k, num_images, batchSize)
      if(count == 0):
        currGradientW, currGradientBias = [], []
        currGradientW = gradientW
        currGradientBias = gradientBias
      else:
        for idx in range(0, len(gradientW)):
          currGradientW[idx] += gradientW[idx] 
          currGradientBias[idx] += gradientBias[idx] 
      count += 1
      if(iterationNumber==epochs-1):
        y_pred.append(np.argmax(activation[num_hidden_layers+1]))
      if count == batchSize:
        count = 0
        W, bias = updateParam(W, currGradientW, bias, currGradientBias, num_hidden_layers+1, eta)
    print(loss/num_images)
  return y_pred, W, bias
    

In [ ]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()
num_hidden_layers = 3
neurons_in_each_layer = 128
num_images = len(trainX)
k=10
trainX.reshape(trainX.shape[0], trainX.shape[1]*trainX.shape[2])
trainX = trainX/255.0
testX = testX/255.0
a, W, bias=stochastic_gradient_descent(num_hidden_layers, neurons_in_each_layer, num_images, 28*28, trainX, trainy, 6, 0.1)

In [28]:
cnt=0
for i in range(len(a)):
  if(a[i]==trainy[i]):
    cnt+=1
print("Accuracy on train data", 100*cnt/len(a))

Accuracy on train data 86.32


In [29]:
count = 0
for i in range(len(testX)):
  activation, preactivation = feedForward(W, bias, testX[i], num_hidden_layers, neurons_in_each_layer)
  y_pred = np.argmax(activation[num_hidden_layers+1])
  if(y_pred == testy[i]):
    count+=1
print("Accuracy on test data", (100.0*count)/len(testy))

Accuracy on test data 84.79
